In [1]:
from Bio import SeqIO
from glob import glob
import os

In [2]:
import numpy
numpy.__version__

'1.22.3'

In [3]:
from mito.genotyping import *

## Load reference

In [4]:
# Path to reference genome
reference_path = '../../data/reference/mito_GRCh38_gimlet.fasta'

with open(reference_path, 'r'):
    reference_seq = next(SeqIO.parse(reference_path, "fasta"))
reference = pd.Series(list(reference_seq.seq))

## Load read matrices

In [5]:
# Path to allele count files
INPUT_PATH = '../../data/P9855_and_bulk_ac/'

In [6]:
cell_count = []

# Get all allele counts files in the input path
cell_count_filenames = list(glob(os.path.join(INPUT_PATH, '*.txt'))) #*.txt

for filename in sorted(cell_count_filenames):
    print('Reading {}'.format(filename))
    name = os.path.basename(filename).split('.')[0]
    

    # Load one allele count file
    counts = pd.read_csv(filename, sep='\t')
    counts.name = name
    cell_count.append(counts)
    
# Get sample order

sample_list = []

for filename in sorted(cell_count_filenames):
    name = os.path.basename(filename).split('-')[0]#.split('_')[-1]
    sample_list.append(name)

sample_list_df = pd.DataFrame(sample_list)
sample_list_df = sample_list_df.rename(columns={0: 'cell'})

Reading ../../data/P9855_and_bulk_ac/P3861_218.clean.dedup_ac.txt
Reading ../../data/P9855_and_bulk_ac/P9855_2085_S108_L004_ac.txt
Reading ../../data/P9855_and_bulk_ac/P9855_2089_S112_L004_ac.txt
Reading ../../data/P9855_and_bulk_ac/P9855_2090_S113_L004_ac.txt
Reading ../../data/P9855_and_bulk_ac/P9855_2091_S114_L004_ac.txt
Reading ../../data/P9855_and_bulk_ac/P9855_2093_S116_L005_ac.txt
Reading ../../data/P9855_and_bulk_ac/P9855_2096_S119_L005_ac.txt
Reading ../../data/P9855_and_bulk_ac/P9855_2101_S124_L006_ac.txt
Reading ../../data/P9855_and_bulk_ac/P9855_2102_S125_L006_ac.txt
Reading ../../data/P9855_and_bulk_ac/P9855_2104_S127_L006_ac.txt
Reading ../../data/P9855_and_bulk_ac/P9855_2110_S133_L007_ac.txt
Reading ../../data/P9855_and_bulk_ac/P9855_2111_S134_L007_ac.txt
Reading ../../data/P9855_and_bulk_ac/P9855_2112_S135_L007_ac.txt


In [8]:
# Obtain a set of error rates to test

e_rates = np.linspace(0.0001, 0.0009, 10).round(4)
extra_error_rates = [0.001, 0.002, 0.005]
extra_error_rates_ar = np.array(extra_error_rates)
#all_e_rates = np.concatenate((e_rates1, extra_error_rates_ar))
all_e_rates = e_rates

## Compute mutations probabilities

In [9]:
def cell_prob(e_rate):
    
    error_rate_when_no_mutation = error_rate_when_mutation = e_rate
    p_mutation = 1 / 500.0

    cell_prob = []
    for count in cell_count:
        count = count.iloc[:reference.shape[0]]  # discard trailing positions
        p = nucleotide_mutation_prob(
            cell_counts=count,
            reference=reference,
            error_rate_when_no_mutation=error_rate_when_no_mutation,
            error_rate_when_mutation=error_rate_when_mutation,
            p_mutation=p_mutation,
        )

        cell_prob.append(p)
        
    # Compute P(mutation | read counts)
    cells_p_mutation = []
    for cell_prob in cell_prob:
        p = mutation_prob(cell_prob, reference)
        cells_p_mutation.append(p)
        
        
    # Make mutation matrix
    mutation_matrix = cells_p_mutation[0][['#CHR', 'POS']].copy()
    mutation_matrix_data = pd.DataFrame(np.dstack([c['Prob_mutation'].values for c in cells_p_mutation]).squeeze())
    mutation_matrix = pd.concat([mutation_matrix, mutation_matrix_data], axis=1)
    return mutation_matrix      

## Filter pmat

In [11]:
def flt_pmat(e_rate):
    mutation_matrix = cell_prob(e_rate)

    # Remove germline mutations
    mutation_matrix = mutation_matrix.loc[mutation_matrix[0] < 0.9]
    
    # Probabilities for location with mutation at any of the cells
    mutation_threshold = 0.9
    data = mutation_matrix.set_index('POS').drop('#CHR', axis=1)
    mutation_any_cell = data[(data > mutation_threshold).any(axis=1)]
    
    # select rows that have a value greater than mutation_threshold in more than x cells
    mutation_threshold = 0.9
    mask = (data.values > mutation_threshold).sum(axis=1) > 1 #create an array with boolean values and sum these along the axis and select rows with more than 1 True
    two_cells_have_mut = data[mask]

    # Fix matrix
    half = len(two_cells_have_mut.columns) / 2

    # Remove rows where 50% of cols are nan and save in new df
    nan_rows = two_cells_have_mut.drop(two_cells_have_mut[(two_cells_have_mut.isna()).sum(axis=1)> half].index)

    # Replace NaN with row mean
    imputed = nan_rows.transpose().fillna(nan_rows.mean(axis=1)).transpose()
    
    # To skip removal of sites with overall high probability for mut
    high_prob_rows = imputed.copy()
    
    # Replace 1.0
    high_prob_rows = high_prob_rows.replace(1.0, 0.99999)
    
    np.set_printoptions(suppress=True)
    e_rate_name = str(e_rate)
    matrix_path = '../../data/P9855_matrix_output/' + e_rate_name + '.csv'
    high_prob_rows.to_csv(matrix_path, index=False, sep=' ', header= False)
    


In [12]:
# Add more error rates
np.set_printoptions(suppress=True)

high_and_low = [0.02, 0.025, 0.03, 0.035, 0.00009, 0.00005]
high_and_low_ar = np.array(high_and_low)

for e_rate in high_and_low_ar:
    flt_pmat(e_rate)

/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  prob_mutation['Prob_mutation'] = 1 - p.lookup(p.index, reference.values)
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  prob_mutation['Prob_mutation'] = 1 - p.lookup(p.index, reference.values)
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  prob_mutation['Prob_mutation'] = 1 - p.lookup(p.index, reference.values)
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecate

/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  prob_mutation['Prob_mutation'] = 1 - p.lookup(p.index, reference.values)
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  prob_mutation['Prob_mutation'] = 1 - p.lookup(p.index, reference.values)
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  prob_mutation['Prob_mutation'] = 1 - p.lookup(p.index, reference.values)
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecate

In [77]:
high_and_low_ar

array([0.02   , 0.025  , 0.03   , 0.035  , 0.00009, 0.00005, 0.00001])

In [54]:
for e_rate in all_e_rates:
    flt_pmat(e_rate)

/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  prob_mutation['Prob_mutation'] = 1 - p.lookup(p.index, reference.values)
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  prob_mutation['Prob_mutation'] = 1 - p.lookup(p.index, reference.values)
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  prob_mutation['Prob_mutation'] = 1 - p.lookup(p.index, reference.values)
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecate

/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  prob_mutation['Prob_mutation'] = 1 - p.lookup(p.index, reference.values)
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  prob_mutation['Prob_mutation'] = 1 - p.lookup(p.index, reference.values)
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  prob_mutation['Prob_mutation'] = 1 - p.lookup(p.index, reference.values)
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecate

/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  prob_mutation['Prob_mutation'] = 1 - p.lookup(p.index, reference.values)
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  prob_mutation['Prob_mutation'] = 1 - p.lookup(p.index, reference.values)
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  prob_mutation['Prob_mutation'] = 1 - p.lookup(p.index, reference.values)
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecate

/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  prob_mutation['Prob_mutation'] = 1 - p.lookup(p.index, reference.values)
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  prob_mutation['Prob_mutation'] = 1 - p.lookup(p.index, reference.values)
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  prob_mutation['Prob_mutation'] = 1 - p.lookup(p.index, reference.values)
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecate

/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  prob_mutation['Prob_mutation'] = 1 - p.lookup(p.index, reference.values)
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  prob_mutation['Prob_mutation'] = 1 - p.lookup(p.index, reference.values)
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  prob_mutation['Prob_mutation'] = 1 - p.lookup(p.index, reference.values)
/Users/johard/Documents/mcmt/MCMT/Notebooks/mito/genotyping.py:251: FutureWarning: The 'lookup' method is deprecate

## Now run mt-SCITE